In [23]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [19]:
from tools.pickle_tools import *
filenames, train_data, train_label = prepare_metadatada_from_pkl_file("Metadata/trainXML.pkl")

PICKLE FILE READED:  Metadata/trainXML.pkl (876, 19)
TRAIN data: (876, 17)
*****	Sample  [0. 0. 0. 0. 0. 0. 3. 7. 1. 2. 3. 0. 3. 7. 1. 0. 0.]
Class: 3 [0. 0. 0. 1.] Name: B5bc8b54ede14c.xml *****


In [20]:
from monumai.monument import Monument
xml_names = [el for sublist in list(Monument.XML_ELEMENT_DIC.values()) for el in sublist]
redundant_features = [(6,12),(7,13),(8,14)]

to_remove = []

for first_occ, second_occ in redundant_features:
    train_data[:,first_occ] += train_data[:,second_occ]
    train_data[:,second_occ] -= train_data[:,second_occ]
    to_remove.append(second_occ)

train_data = np.delete(train_data, to_remove, axis=1)

In [21]:
names = [el for sublist in list(Monument.ELEMENT_DIC.values()) for el in sublist]

threshold = 0.3

train_data[train_data<threshold] = 0
train_data[train_data >= threshold] = 1

train_data = train_data.astype(np.uint8)

train_names = [ ]
    
for feature_vec in train_data:
    feature_vec_names = []
    for i in range(len(feature_vec)):
        if feature_vec[i]:
            feature_vec_names.append(names[i])
    train_names.append(feature_vec_names)

In [24]:
te = TransactionEncoder()
te_ary = te.fit(train_names).transform(train_names)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [25]:
df

,arco-apuntado,arco-conopial,arco-herradura,arco-lobulado,arco-medio-punto,arco-trilobulado,columna-salomonica,dintel-adovelado,fronton,fronton-curvo,fronton-partido,ojo-de-buey,serliana,vano-adintelado
0,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,True,True,True,False
2,False,False,False,False,True,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,False,False,False,True
4,True,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,False,False,False,False,True,False,False,False,True,False,False,False,False,False
872,False,False,False,False,True,False,False,False,True,False,True,False,False,False
873,False,False,False,False,True,False,False,False,False,True,True,False,False,False
874,False,False,False,False,False,False,False,False,True,False,True,False,False,False


In [26]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[ (frequent_itemsets['length']>= 2) &
                   (frequent_itemsets['support'] >= 0.05) ]

,support,itemsets,length
18,0.068493,"(dintel-adovelado, arco-herradura)",2
19,0.118721,"(fronton, arco-medio-punto)",2
20,0.149543,"(arco-medio-punto, fronton-curvo)",2
21,0.109589,"(fronton-partido, arco-medio-punto)",2
28,0.097032,"(fronton, fronton-curvo)",2
29,0.138128,"(fronton-partido, fronton)",2
31,0.052511,"(fronton, serliana)",2
33,0.154110,"(fronton-partido, fronton-curvo)",2
35,0.051370,"(serliana, fronton-curvo)",2
36,0.067352,"(vano-adintelado, fronton-curvo)",2


In [27]:
frequent_itemsets['itemsets']

0                                       (arco-apuntado)
1                                       (arco-conopial)
2                                      (arco-herradura)
3                                       (arco-lobulado)
4                                    (arco-medio-punto)
                            ...                        
57    (fronton-partido, fronton, vano-adintelado, ar...
58    (fronton-partido, arco-medio-punto, vano-adint...
59    (arco-medio-punto, ojo-de-buey, fronton-curvo,...
60    (fronton-partido, fronton, vano-adintelado, fr...
61    (vano-adintelado, fronton-curvo, fronton-parti...
Name: itemsets, Length: 62, dtype: object